![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

## Spectroscope

#### August 31, 2022

Based on Oscilloscope 7.

Let's just view the FFT info instead. Note the values are given in Decibels, so we have to adjust the limits on our Plotly graph. 

I can set the sample rate in the JS code, and use this to figure out the axis range for the frequencies. 

Amplitude is in decibels. 


In [1]:
import plotly.graph_objects as go
from IPython.display import Javascript


In [2]:
## Let's set up the frequency lines, Sample Rate and FFT size have to be the same as in the Javascript code
from numpy import linspace
sampleRate = 10000
FFTsize = 256
x = linspace(0,sampleRate//2,FFTsize//2)

In [3]:
myDataArray = x

fig = go.Figure(data=go.Scatter( x=x,y=x, mode='lines'))
fig.update_layout(yaxis_range=[-120,0],title="Spectrogram",xaxis_title="Frequency",yaxis_title="Magnitude (dB)")
f = go.FigureWidget(fig)
f

FigureWidget({
    'data': [{'mode': 'lines',
              'type': 'scatter',
              'uid': '035124b8-…

In [4]:
Javascript("""

// We use the WebAudio API in Javascript in order to access the user's microphone. It asks for permission first. 

// Some variable to keep track of the audio context, source, and stream

var audioCtx = new (window.AudioContext || window.webkitAudioContext)({sampleRate: 10000,});
var source;
var stream;

//set up the analyser node that grabs the sound samples

var analyser = audioCtx.createAnalyser();
    analyser.minDecibels = -90;
    analyser.maxDecibels = -10;
    analyser.smoothingTimeConstant = 0.85;

// a variable to keep track of animation frames

var drawVisual;

// start up the microphone (media) and then connect it to the visualizer

if (navigator.mediaDevices.getUserMedia) {
    console.log("mediaDevices.getUserMedia supported.");
    async function getMedia(constraints) {
        try {
            stream = await navigator.mediaDevices.getUserMedia(constraints);
            source = audioCtx.createMediaStreamSource(stream);
            source.connect(analyser);
            visualize();
        } 
        catch (err) {
            console.log("The following gUM error occured: " + err);
        }
    }
    getMedia({audio: true,})
} else {
          console.log("mediaDevices.getUserMedia not supported on your browser!");
};

// This visualize function catches the sound samples then passes it to the Plotly device in Python
// We also check one global variable "killDraw" to see when we should stop the drawing. 
function visualize() {
    analyser.fftSize = 256;
    var dataArray = new Float32Array(analyser.frequencyBinCount);
    var fps = 4;
    window.killDraw = false;
    function draw(timestamp){
        if (window.killDraw) {return;}
        setTimeout(function(){ //throttle requestAnimationFrame to 4 fps
            analyser.getFloatFrequencyData(dataArray);
            IPython.notebook.kernel.execute(
                "f.data[0]['y'] = "+ dataArray + ";"
            ); 
            drawVisual = requestAnimationFrame(draw)
        }, 1000/fps)
    }
    draw();
}

// finally, we save the "visualize" function in a variable we can call from a Jupyter cell later. 
window.myVis = visualize;

""")

<IPython.core.display.Javascript object>

## Starting and stopping

We include two cells to start and stop the drawing. Otherwise, the drawing is running all the time and never stops working. 

In [5]:
## Run this cell to stop the drawing
Javascript("window.killDraw=true;")

<IPython.core.display.Javascript object>

In [6]:
## Run this cell to restart the drawing
Javascript("window.killDraw=true; window.myVis();")

<IPython.core.display.Javascript object>

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)

In [13]:
%%js
window.killDraw = true;
console.log(window.killDraw)

<IPython.core.display.Javascript object>

In [10]:
myDataArray

[0, 0.1, 0.2, 0.3]

In [11]:
f.data[0]['y']

(-72.69232177734375,
 -73.7354965209961,
 -80.48992156982422,
 -88.05966186523438,
 -89.74690246582031,
 -91.5738296508789,
 -94.54830169677734,
 -95.40719604492188,
 -97.6620101928711,
 -97.91693878173828,
 -93.91902160644531,
 -93.01429748535156,
 -94.64923095703125,
 -94.25202941894531,
 -95.08991241455078,
 -97.97381591796875,
 -96.54663848876953,
 -96.14773559570312,
 -96.80858612060547,
 -98.14303588867188,
 -103.04718780517578,
 -100.89405822753906,
 -98.60954284667969,
 -100.12976837158203,
 -100.1986083984375,
 -98.70745849609375,
 -96.6347427368164,
 -97.5508041381836,
 -102.12762451171875,
 -99.94876861572266,
 -98.14834594726562,
 -98.86386108398438,
 -96.61235046386719,
 -96.06211853027344,
 -97.23076629638672,
 -97.9860610961914,
 -99.43858337402344,
 -99.94490814208984,
 -101.7662353515625,
 -102.26920318603516,
 -102.12067413330078,
 -103.76023864746094,
 -101.8980484008789,
 -101.0054702758789,
 -102.11311340332031,
 -101.53770446777344,
 -101.34312438964844,
 -98.5931

In [30]:
3/2


1.5

In [32]:
4//2

2